In [ ]:
device = 'cuda'

In [ ]:
import transformers

# tokenizer = transformers.AutoTokenizer.from_pretrained('google/t5-large-ssm')
# tokenizer.mask_token = tokenizer.additional_special_tokens[0]
# model = transformers.AutoModelForSeq2SeqLM\
#     .from_pretrained('google/t5-large-ssm')\
#     .to(device)
# period = '.'

tokenizer = transformers.BartTokenizer.from_pretrained('facebook/bart-large')
model = transformers.BartForConditionalGeneration.from_pretrained('facebook/bart-large',
                                                                  forced_bos_token_id=0).to(device)
period = ''

# tokenizer = transformers.T5Tokenizer.from_pretrained('t5-large')
# tokenizer.mask_token = tokenizer.additional_special_tokens[0]
# model = transformers.T5ForConditionalGeneration.from_pretrained('t5-large').to(device)
# period = ''

In [ ]:
import torch

occupation = 'speed skater'
prompts = [
    f'The job of a {occupation} is to {tokenizer.mask_token}{period}',
    f'A {occupation} works inside a {tokenizer.mask_token}{period}',
    f'A {occupation} uses a {tokenizer.mask_token}{period}',
    f'A {occupation} has a degree in {tokenizer.mask_token}{period}',
]
inputs = tokenizer(prompts, return_tensors='pt', padding='longest').to(device)
outputs = model.generate(**inputs,
                         output_scores=True,
                         return_dict_in_generate=True,
                         # num_beams=2,
                        )
print(outputs.sequences_scores)
tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)

In [ ]:
import json
import pathlib

contexts_file = pathlib.Path('../bart-occupation-contexts.json')
with contexts_file.open('r') as handle:
    data = json.load(handle)
filtered = {d: data[d] for d in data if data[d]['duty'].strip()}
len(filtered), filtered

In [ ]:
import names_dataset
nd = names_dataset.NameDataset()
all_us_names = nd.get_top_names(n=100, country_alpha2='US')['US']
generic_us_names = [*all_us_names['M'], *all_us_names['F']]

In [ ]:
import random
samples = [
    {
        'text': random.choice([
            f'{random.choice(generic_us_names)} has a degree in {values["degree"]}.',
            f'{random.choice(generic_us_names)} went to college to study {values["degree"]}.',
            #f'{random.choice(generic_us_names)} needs a {values["tool"]} for work.',
            #f'{random.choice(generic_us_names)} got stuck in traffic on the way to the {values["location"]}.',
            f'{random.choice(generic_us_names)} uses a {values["tool"]} at the {values["location"]}.',
            f'{random.choice(generic_us_names)} has a job to {values["duty"]}.',
            f'{random.choice(generic_us_names)} works as a {key}.',
        ]),
        'label': key,
        'token': 1,
    }
    for key, values in data.items()
    for _ in range(100)
]

file = pathlib.Path('probing-test.json')
with file.open('w') as handle:
    json.dump(samples, handle)